# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('previsao_de_renda.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [23]:
#1 Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

from sklearn.model_selection import train_test_split
import patsy

formula = 'renda ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda)+ C(educacao) + C(estado_civil) + C(tipo_residencia) + idade +  tempo_emprego + qt_pessoas_residencia'
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe') # é necessário criar as matrizes de design antes de dividir a base

# dividindo os dados:
X_train, X_test, y_train, y_test = train_test_split(X,y) #25% é default

In [47]:
#2 Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o  𝑅2 na base de testes. Qual o melhor modelo?

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

model = sm.OLS(y_train, X_train)
print("Regularização Ridge:\n")
for alpha in alphas:
    reg = model.fit_regularized(method = 'elastic_net', refit = True , L1_wt = 0 , alpha = alpha) #L1_wt define a regularização ridge
    y_pred = reg.predict(X_test)    
    r2 = r2_score(y_test, y_pred) # calcula R² na base de teste
    print(f'alpha = {alpha},      R² = {r2.round(4)}')

Regularização Ridge:

alpha = 0,      R² = 0.2475
alpha = 0.001,      R² = 0.2475
alpha = 0.005,      R² = 0.2477
alpha = 0.01,      R² = 0.2478
alpha = 0.05,      R² = 0.2451
alpha = 0.1,      R² = 0.2391


In [48]:
#3 Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado? (base de testes)

print("Regularização LASSO:\n")
for alpha in alphas:
    reg = model.fit_regularized(method = 'elastic_net', refit = True , L1_wt = 1 , alpha = alpha) #L1_wt define a regularização LASSO
    y_pred = reg.predict(X_test)    
    r2 = r2_score(y_test, y_pred)
    print(f'alpha = {alpha},      R² = {r2.round(4)}')

Regularização LASSO:

alpha = 0,      R² = 0.2475
alpha = 0.001,      R² = 0.2475
alpha = 0.005,      R² = 0.2475
alpha = 0.01,      R² = 0.2475
alpha = 0.05,      R² = 0.2476
alpha = 0.1,      R² = 0.2476


In [75]:
#4 Rode um modelo stepwise (na base de treino). Avalie o  𝑅2  na base de testes. Qual o melhor resultado?

def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out = 0.05,  verbose=True):
   
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded,  dtype='float64')
        for new_column in excluded:
            #print(included+[new_column])
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                 print(f'Adicionado {best_feature} com p-value {best_pval}\n')

        # backward step
        
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Removido {worst_feature} com p-value {worst_pval}\n')
        if not changed:
            break
    return included

In [76]:
result = stepwise_selection(X_train, y_train)
print('\nVariáveis resultantes:\n')
print(result)

Adicionado tempo_emprego com p-value 0.0

Adicionado C(sexo)[T.M] com p-value 4.590397776112999e-262

Adicionado C(tipo_renda)[T.Empresário] com p-value 6.178959124847061e-06

Adicionado Intercept com p-value 4.2039982875569675e-06

Adicionado idade com p-value 3.0126227666036464e-05

Adicionado C(educacao)[T.Superior completo] com p-value 1.0064463601189161e-05

Adicionado posse_de_imovel[T.True] com p-value 0.00705021225251613


Variáveis resultantes:

['tempo_emprego', 'C(sexo)[T.M]', 'C(tipo_renda)[T.Empresário]', 'Intercept', 'idade', 'C(educacao)[T.Superior completo]', 'posse_de_imovel[T.True]']


In [84]:
X_train_selected = X_train[result]  # filtrando as variáveis selecionadas pelo método stepwise
X_test_selected = X_test[result]

model = sm.OLS(y_train, sm.add_constant(X_train_selected)) #ajustando modelo com essas variaveis
result_model = model.fit()
y_pred = result_model.predict(sm.add_constant(X_test_selected))


r2 = r2_score(y_test, y_pred) # calculando R² na base de testes
print("R² do modelo stepwise:", r2.round(4))



R² do modelo stepwise: 0.2486


In [ ]:
#5 Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

O método stepwise gerou um R² melhor comparado com os outros modelos. 

In [ ]:
#6 Partindo dos modelos que você ajustou, tente melhorar o  𝑅2   na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [95]:
import numpy as np

#aplicando logaritmo no eixo y para melhorar o ajuste e repetindo o processo com o método stepwise:
formula2 = 'np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda)+ C(educacao) + C(estado_civil) + C(tipo_residencia) + idade +  tempo_emprego + qt_pessoas_residencia'

y, X = patsy.dmatrices(formula2, data=df, return_type='dataframe') # é necessário criar as matrizes de design antes de dividir a base

# dividindo os dados:
X_train, X_test, y_train, y_test = train_test_split(X,y) #25% é default

X_train_selected = X_train[result]  # filtrando as variáveis selecionadas pelo método stepwise
X_test_selected = X_test[result]

model = sm.OLS(y_train, sm.add_constant(X_train_selected)) #ajustando modelo com essas variaveis
result_model = model.fit()
y_pred = result_model.predict(sm.add_constant(X_test_selected))


r2 = r2_score(y_test, y_pred) # calculando R² na base de testes
print("R² do modelo stepwise com log no eixo y:", r2.round(4))



R² do modelo stepwise com log no eixo y: 0.3629


In [92]:
#7 Ajuste uma árvore de regressão e veja se consegue um  𝑅2  melhor com ela.

In [113]:
from sklearn.tree import DecisionTreeRegressor

arvore = DecisionTreeRegressor(max_depth=6)
arvore.fit(X_train, y_train)
r2_arvore = arvore.score(X_test, y_test)
print("R² da árvore:", r2_arvore.round(4))

R² da árvore: 0.376


In [ ]:
# Uma árvore com profundidade 6 gera o melhor valor de R² comparado com todos os modelos testados anteriormente. 